In [2]:
import sys
import os
import pandas as pd
# 현재 작업 디렉토리 기준으로 src 디렉토리 추가
notebook_dir = os.getcwd()  # 현재 Jupyter Notebook의 작업 디렉토리
# project_root = os.path.abspath(os.path.dirname(notebook_dir))
sys.path.append(os.path.dirname(notebook_dir))
# project_root

# tools 모

In [ ]:
from src.tools.google_drive import ensure_data_files

# 데이터 파일 경로 가져오기
data_paths = ensure_data_files()
print(data_paths)


In [ ]:
data_paths

In [ ]:
# load data
diner = pd.read_csv(data_paths["diner"])
diner_category = pd.read_csv(data_paths["category"])
diner = pd.merge(diner, diner_category, on="diner_idx", how="left")


review = pd.read_csv(data_paths["review"])
reviewer = pd.read_csv(data_paths["reviewer"])
review = pd.merge(review, reviewer, on="reviewer_id", how="left")

In [ ]:
diner_lat_lon = diner[diner['diner_num_address'].str.contains('마포구', na=False)]
len(diner_lat_lon)

In [ ]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic

# 셀 크기 정의 (예: 0.01도 단위로 셀 나누기)
cell_size = 0.01

# 지역별로 셀을 나누기 위해 그룹화
def assign_cell(lat, lon, cell_size):
    lat_cell = np.floor(lat / cell_size) * cell_size
    lon_cell = np.floor(lon / cell_size) * cell_size
    return f"{lat_cell:.5f}_{lon_cell:.5f}"

diner_lat_lon["cell"] = diner_lat_lon.apply(lambda row: assign_cell(row["diner_lat"], row["diner_lon"], cell_size), axis=1)

# 각 셀의 중심 좌표 계산
cell_centers = diner_lat_lon.groupby("cell").agg(
    center_lat=("diner_lat", "mean"),
    center_lon=("diner_lon", "mean")
).reset_index()

# 결과 출력
cell_centers

In [ ]:
diner_lat_lon

## 1. K-means 클러스터링

In [ ]:
diner_lat_lon.dropna(subset=['diner_lat', 'diner_lon', 'diner_num_address'], inplace=True)

In [ ]:
from sklearn.preprocessing import LabelEncoder
diner_lat_lon['address_short'] = diner_lat_lon['diner_num_address'].apply(lambda x: " ".join(x.split()[:3]))
le = LabelEncoder()
diner_lat_lon['address_short_idx'] = le.fit_transform(diner_lat_lon['address_short'])



In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.cluster import KMeans

# coords = coords_m[1:]  # 미터 단위 좌표값 사용
coords = diner_lat_lon[['diner_lat', 'diner_lon']].values

sse = []
k_range = range(5, 51, 5)  # 5~50까지 테스트 (너무 세부적이면 과도하게 많음)
for k in k_range:
    km = KMeans(n_clusters=k, random_state=42, n_init='auto')
    km.fit(coords)
    sse.append(km.inertia_)

plt.figure(figsize=(10, 6))
plt.plot(k_range, sse, 'bx-')
plt.xlabel('Number of clusters (k)')
plt.ylabel('SSE (Sum of squared errors)')
plt.title('Elbow Method For Optimal k')
plt.show()

In [ ]:
# 시군구 별 20개 * 시군구 개수(서울 25개)
best_k = 500  # 예시값 (위의 Elbow Method에서 정한 값)

kmeans = KMeans(n_clusters=best_k, random_state=42, n_init='auto')
diner_lat_lon['cluster'] = kmeans.fit_predict(coords)

In [ ]:
diner_lat_lon['cluster'].value_counts()

In [17]:
import folium
from folium.plugins import MarkerCluster

# 클러스터 라벨 컬럼이 있다고 가정
import random
colors = [f'#{random.randint(0, 0xFFFFFF):06x}' for _ in range(diner_lat_lon['cluster'].nunique())]

import folium
m = folium.Map(location=[37.55, 126.98], zoom_start=11)

for idx, row in diner_lat_lon.iterrows():
    folium.CircleMarker(
        location=[row['diner_lat'], row['diner_lon']],
        radius=3,
        color=f"{colors[row['cluster']]}",
        fill=True,
        fill_opacity=0.7,
        popup=row['diner_name']
    ).add_to(m)


In [ ]:
import h3
import folium
import random

# 원하는 해상도 설정 (9 추천: 약 170m 단위)
resolution = 9

# H3 인덱스 컬럼 생성
diner_lat_lon['h3_index'] = diner_lat_lon.apply(
    lambda x: h3.latlng_to_cell(x['diner_lat'], x['diner_lon'], resolution), axis=1
)

# 클러스터와 H3를 조합하여 상업권 그룹핑 가능
diner_lat_lon['cluster_h3'] = diner_lat_lon['cluster'].astype(str) + "_" + diner_lat_lon['h3_index']

# 간단히 확인
print(diner_lat_lon[['diner_name', 'cluster', 'h3_index', 'cluster_h3']].head())

In [ ]:
import h3
import folium
import random

# 원하는 해상도 (추천: 9 → 약 170m 단위)
resolution = 9

# H3 인덱스 컬럼 추가 (최신 API 사용)
diner_lat_lon['h3_index'] = diner_lat_lon.apply(
    lambda x: h3.latlng_to_cell(x['diner_lat'], x['diner_lon'], resolution), axis=1
)

# 클러스터 및 H3 인덱스 조합
diner_lat_lon['cluster_h3'] = diner_lat_lon['cluster'].astype(str) + "_" + diner_lat_lon['h3_index']

# Folium 시각화
m = folium.Map(location=[37.55, 126.98], zoom_start=13)

# 클러스터별 색상
clusters = diner_lat_lon['cluster'].unique()
colors = {label: f'#{random.randint(0, 0xFFFFFF):06x}' for label in clusters}

# H3 육각형 시각화
for h3_index in diner_lat_lon['h3_index'].unique():
    boundary = h3.cell_to_boundary(h3_index)  # 최신 API 사용
    cluster = diner_lat_lon[diner_lat_lon['h3_index'] == h3_index]['cluster'].mode()[0]

    folium.Polygon(
        locations=boundary,
        color=colors[cluster],
        fill=True,
        fill_opacity=0.3,
        popup=f'Cluster: {cluster}\nH3: {h3_index}'
    ).add_to(m)

# 음식점 위치 표시 (선택사항)
# for _, row in diner_lat_lon.iterrows():
#     folium.CircleMarker(
#         [row['diner_lat'], row['diner_lon']],
#         radius=2,
#         color='black',
#         fill=True,
#         fill_opacity=0.7,
#         popup=row['diner_name']
#     ).add_to(m)

m

In [ ]:
diner_lat_lon['cluster_h3']

In [20]:
m.save('h3_cluster_map_updated.html')